##### Define Pages

In [1]:
source_detail = "https://www.ird.gov.hk/charity/view_detail.php"
source_root = "https://www.ird.gov.hk/eng/tax/ach_index.htm"
source_search = "https://www.ird.gov.hk/charity/search_result.php"

##### Import Dependencies

In [2]:
import requests

In [3]:
def search_page(term):
    data = {"q": str(term)}
    return requests.post(source_search, data)

##### Search with query "crossroads"

In [4]:
r = search_page("crossroads")

In [7]:
html = r.text
# html

In [11]:
from bs4 import BeautifulSoup as bs

soup = bs(html, 'html.parser')
body = soup.body

results = body.find(id='search_des')
print(results.text)

Search result for "crossroads": 5 record(s) found搜尋結果 "crossroads": 共找到5個記錄


In [9]:
# print(bs(str(body), "html.parser").prettify())

In [10]:
orgs = body.find_all("form")
print(orgs)

[<form action="view_detail.php" id="myform_0" method="POST" onsubmit="target_popup(this)"><input name="org_id" type="hidden" value="S010351"/><a class="mylink" href="#">Crossroads' Global X-perience<br/>環球 X 體驗</a></form>, <form action="view_detail.php" id="myform_1" method="POST" onsubmit="target_popup(this)"><input name="org_id" type="hidden" value="91/04820"/><a class="mylink" href="#">CROSSROADS FOUNDATION LIMITED<br/>國際十字路協會有限公司</a></form>, <form action="view_detail.php" id="myform_2" method="POST" onsubmit="target_popup(this)"><input name="org_id" type="hidden" value="S006486"/><a class="mylink" href="#">Crossroads Global Distribution<br/>十字路會環球分派中心</a></form>, <form action="view_detail.php" id="myform_3" method="POST" onsubmit="target_popup(this)"><input name="org_id" type="hidden" value="S008168"/><a class="mylink" href="#">Crossroads Global Handicrafts<br/>十字路會環球工藝村</a></form>, <form action="view_detail.php" id="myform_4" method="POST" onsubmit="target_popup(this)"><input name

In [68]:
# tags = body.find_all("a", class_="mylink")


# names = []
# for tag in tags:
#     string = tag.encode_contents().replace(b'<br/>', b'\t')
#     text = bs(string, "html.parser").text
#     names.append(text)

# names